In [40]:
import numpy as np
import pandas as pd
import sklearn.ensemble
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
plt.style.use('ggplot')
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test  

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().dro

In [3]:
gbm = GradientBoostingRegressor()
gbm.set_params(learning_rate = 0.01, n_estimators = 500, subsample = 0.7)

gbm.fit(X_train,Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(learning_rate=0.01, n_estimators=500, subsample=0.7)

In [4]:
# print(gbm.best_score_)
print(gbm.score(X_train,Y_train))
print(gbm.score(X_test,Y_test))

0.990701403636799
0.9887893753897119


In [5]:
pred = gbm.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = gbm.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)



print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print(train_RMSE)
print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))

Train RMSE =  67.93430949980913
67.93430949980913
Test RMSE =  78.90775111339609


In [6]:
# GradientBoostingRegressor(learning_rate=0.01, n_estimators=1000, subsample=0.5)

# grid_para_tree = [{'learning_rate': [0.01,0.02,0.03,0.04],
#                   'subsample'    : [0.9, 0.5, 0.2, 0.1],
#                   'n_estimators' : [50,100,250,500,1000]
# #                   'max_depth'    : [4,6,8,10]

                   
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [100, 500, 1000]
#                  'max_depth'    : [2,5,10]
                 }]

boost = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 25)

boost.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=25, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 500, 1000],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [7]:
boost.best_estimator_

GradientBoostingRegressor(learning_rate=0.015, n_estimators=500, subsample=0.4)

In [8]:
print(boost.best_score_)
print(boost.score(X_test,Y_test))

0.9840161286190025
0.9892429191539951


In [29]:
pred = boost.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = boost.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print("Train RMSE = ", train_RMSE)
test_RMSE = np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog))
print("Test RMSE = ", test_RMSE)

Train RMSE =  62.66889254523786
Test RMSE =  76.32162797878041


In [30]:
# Create a dictionary of the results
r2_train = boost.best_score_
r2_test = boost.best_estimator_.score(X_test, Y_test)
rf_output = pd.DataFrame()
rf_output['All'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All
RMSE_train,62.668893
RMSE_test,76.321628
R2_train,0.984016
R2_test,0.989243


## Same for all Clusters

In [10]:
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4],
                  'n_estimators' : [100, 250, 500, 750]
#                   'max_depth'    : [4,6,8,10]
                 }]


In [11]:
boost1 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 25)

boost1.fit(c1_train, Y_c1train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=25, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 250, 500, 750],
                                         'subsample': [0.6, 0.5, 0.4]}])

In [12]:
boost2 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 25)

boost2.fit(c2_train, Y_c2train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=25, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 250, 500, 750],
                                         'subsample': [0.6, 0.5, 0.4]}])

In [13]:
boost3 = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 25)

boost3.fit(c3_train, Y_c3train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=25, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 250, 500, 750],
                                         'subsample': [0.6, 0.5, 0.4]}])

## Results of Gradient Boost Clusters:

In [31]:
# Results of Cluster 1:
print("R2 of train = ", boost1.best_score_)
print("R2 of test = ", boost1.best_estimator_.score(c1_test, Y_c1test))

pred1 = boost1.predict(c1_train)
pred_unlog1 = np.exp(pred1)

pred_test1 = boost1.predict(c1_test)
pred_test_unlog1 = np.exp(pred_test1)

Y_unlog1 = np.exp(Y_c1train)
Y_test_unlog1 = np.exp(Y_c1test)

train_RMSE1 = mean_squared_error(Y_unlog1, pred_unlog1, squared = False)
test_RMSE1 = np.sqrt(mean_squared_error(pred_test_unlog1, Y_test_unlog1))
print("Train RMSE = ", train_RMSE1)
print("Test RMSE = ", test_RMSE1)

R2 of train =  0.96798707864737
R2 of test =  0.9864775761244502
Train RMSE =  50.65400089637982
Test RMSE =  79.88759423380877


In [33]:
r2_train = boost1.best_score_
r2_test = boost1.best_estimator_.score(c1_test, Y_c1test)
rf_output['Cluster_1'] = [train_RMSE1, test_RMSE1, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1
RMSE_train,62.668893,50.654001
RMSE_test,76.321628,79.887594
R2_train,0.984016,0.967987
R2_test,0.989243,0.986478


In [34]:
# Results of Cluster 2:
print("R2 of train = ", boost2.best_score_)
print("R2 of test = ", boost2.best_estimator_.score(c2_test, Y_c2test))

pred2 = boost2.predict(c2_train)
pred_unlog2 = np.exp(pred2)

pred_test2 = boost2.predict(c2_test)
pred_test_unlog2 = np.exp(pred_test2)

Y_unlog2 = np.exp(Y_c2train)
Y_test_unlog2 = np.exp(Y_c2test)

train_RMSE2 = mean_squared_error(Y_unlog2, pred_unlog2, squared = False)
test_RMSE2 = np.sqrt(mean_squared_error(pred_test_unlog2, Y_test_unlog2))
print("Train RMSE = ", train_RMSE2)
print("Test RMSE = ", test_RMSE2)

R2 of train =  0.9813816529341172
R2 of test =  0.9858426291836991
Train RMSE =  52.465480690459735
Test RMSE =  72.38318204449698


In [35]:
r2_train = boost2.best_score_
r2_test = boost2.best_estimator_.score(c2_test, Y_c2test)
rf_output['Cluster_2'] = [train_RMSE2, test_RMSE2, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2
RMSE_train,62.668893,50.654001,52.465481
RMSE_test,76.321628,79.887594,72.383182
R2_train,0.984016,0.967987,0.981382
R2_test,0.989243,0.986478,0.985843


In [16]:
# Results of Cluster 3:
print("R2 of train = ", boost3.best_score_)
print("R2 of test = ", boost3.best_estimator_.score(c3_test, Y_c3test))

pred3 = boost3.predict(c3_train)
pred_unlog3 = np.exp(pred3)

pred_test3 = boost3.predict(c3_test)
pred_test_unlog3 = np.exp(pred_test3)

Y_unlog3 = np.exp(Y_c3train)
Y_test_unlog3 = np.exp(Y_c3test)

train_RMSE3 = mean_squared_error(Y_unlog3, pred_unlog3, squared = False)
test_RMSE3 = np.sqrt(mean_squared_error(pred_test_unlog3, Y_test_unlog3))
print("Train RMSE = ", train_RMSE3)
print("Test RMSE = ", test_RMSE3)

R2 of train =  0.9670208465728874
R2 of test =  0.9840483138186662
Train RMSE =  50.078748214801706
Test RMSE =  85.51817752798759


In [36]:
r2_train = boost3.best_score_
r2_test = boost3.best_estimator_.score(c3_test, Y_c3test)
rf_output['Cluster_3'] = [train_RMSE3, test_RMSE3, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2,Cluster_3
RMSE_train,62.668893,50.654001,52.465481,50.078748
RMSE_test,76.321628,79.887594,72.383182,85.518178
R2_train,0.984016,0.967987,0.981382,0.967021
R2_test,0.989243,0.986478,0.985843,0.984048


## Create Pickle Files for boosting models

In [38]:
# Create pickle files for all clusters:
import pickle
pickle.dump(boost, open('boost_all.pkl','wb'))
pickle.dump(boost1, open('boost_C1.pkl','wb'))
pickle.dump(boost2, open('boost_C2.pkl','wb'))
pickle.dump(boost3, open('boost_C3.pkl','wb'))

# Gradient Boosting with PCA

## Run PCA

In [41]:
# Run a PCA on all data:
scaler_pca = StandardScaler().fit(X_train)
features = scaler_pca.transform(X_train)
df1_scal = pd.DataFrame(features, columns = X_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df1_scal)

df_reduced = pd.DataFrame(pca.components_, columns = X_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat.append(ff)

feat.append('log_value')
X_PCA_train = X_train[feat]
X_PCA_test = X_test[feat]

## All Data with PCA

In [42]:
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [100, 500, 1000]
#                  'max_depth'    : [2,5,10]
                 }]

boost_pca = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 50)

boost_pca.fit(X_PCA_train, Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 48 is smaller than n_iter=50. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 500, 1000],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [43]:
pred = boost_pca.predict(X_PCA_train)
pred_unlog = np.exp(pred)

pred_test = boost_pca.predict(X_PCA_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print("Train RMSE = ", train_RMSE)
test_RMSE = np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog))
print("Test RMSE = ", test_RMSE)

Train RMSE =  63.59608613572196
Test RMSE =  75.95599285849039


In [44]:
r2_train = boost_pca.best_score_
r2_test = boost_pca.best_estimator_.score(X_PCA_test, Y_test)
rf_output['PCA_All'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2,Cluster_3,PCA_All
RMSE_train,62.668893,50.654001,52.465481,50.078748,63.596086
RMSE_test,76.321628,79.887594,72.383182,85.518178,75.955993
R2_train,0.984016,0.967987,0.981382,0.967021,0.984276
R2_test,0.989243,0.986478,0.985843,0.984048,0.989537


## Cluster 1 with PCA

In [45]:
c1_pca_train = c1_train[feat]
c1_pca_test = c1_test[feat]
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [100, 500, 1000]
#                  'max_depth'    : [2,5,10]
                 }]

boost1_pca = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 48)

boost1_pca.fit(c1_pca_train, Y_c1train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=48, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 500, 1000],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [46]:
# Results of Cluster 1:
print("R2 of train = ", boost1_pca.best_score_)
print("R2 of test = ", boost1_pca.best_estimator_.score(c1_pca_test, Y_c1test))

pred1 = boost1_pca.predict(c1_pca_train)
pred_unlog1 = np.exp(pred1)

pred_test1 = boost1_pca.predict(c1_pca_test)
pred_test_unlog1 = np.exp(pred_test1)

Y_unlog1 = np.exp(Y_c1train)
Y_test_unlog1 = np.exp(Y_c1test)

train_RMSE1 = mean_squared_error(Y_unlog1, pred_unlog1, squared = False)
test_RMSE1 = np.sqrt(mean_squared_error(pred_test_unlog1, Y_test_unlog1))
print("Train RMSE = ", train_RMSE1)
print("Test RMSE = ", test_RMSE1)

R2 of train =  0.9699556143587911
R2 of test =  0.9867038193349225
Train RMSE =  57.51931796482435
Test RMSE =  79.22576330279492


In [47]:
r2_train = boost1_pca.best_score_
r2_test = boost1_pca.best_estimator_.score(c1_pca_test, Y_c1test)
rf_output['PCA_Cluster_1'] = [train_RMSE1, test_RMSE1, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2,Cluster_3,PCA_All,PCA_Cluster_1
RMSE_train,62.668893,50.654001,52.465481,50.078748,63.596086,57.519318
RMSE_test,76.321628,79.887594,72.383182,85.518178,75.955993,79.225763
R2_train,0.984016,0.967987,0.981382,0.967021,0.984276,0.969956
R2_test,0.989243,0.986478,0.985843,0.984048,0.989537,0.986704


## Cluster 2 with PCA

In [48]:
c2_pca_train = c2_train[feat]
c2_pca_test = c2_test[feat]
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [100, 500, 1000]
#                  'max_depth'    : [2,5,10]
                 }]

boost2_pca = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 48)

boost2_pca.fit(c2_pca_train, Y_c2train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=48, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 500, 1000],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [49]:
# Results of Cluster 1:
print("R2 of train = ", boost2_pca.best_score_)
print("R2 of test = ", boost2_pca.best_estimator_.score(c2_pca_test, Y_c2test))

pred2 = boost2_pca.predict(c2_pca_train)
pred_unlog2 = np.exp(pred2)

pred_test2 = boost2_pca.predict(c2_pca_test)
pred_test_unlog2 = np.exp(pred_test2)

Y_unlog2 = np.exp(Y_c2train)
Y_test_unlog2 = np.exp(Y_c2test)

train_RMSE2 = mean_squared_error(Y_unlog2, pred_unlog2, squared = False)
test_RMSE2 = np.sqrt(mean_squared_error(pred_test_unlog2, Y_test_unlog2))
print("Train RMSE = ", train_RMSE2)
print("Test RMSE = ", test_RMSE2)

R2 of train =  0.9815825413165492
R2 of test =  0.985765688673127
Train RMSE =  55.018928453710416
Test RMSE =  72.56541887192918


In [50]:
r2_train = boost2_pca.best_score_
r2_test = boost2_pca.best_estimator_.score(c2_pca_test, Y_c2test)
rf_output['PCA_Cluster_2'] = [train_RMSE2, test_RMSE2, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2,Cluster_3,PCA_All,PCA_Cluster_1,PCA_Cluster_2
RMSE_train,62.668893,50.654001,52.465481,50.078748,63.596086,57.519318,55.018928
RMSE_test,76.321628,79.887594,72.383182,85.518178,75.955993,79.225763,72.565419
R2_train,0.984016,0.967987,0.981382,0.967021,0.984276,0.969956,0.981583
R2_test,0.989243,0.986478,0.985843,0.984048,0.989537,0.986704,0.985766


## Cluster 3 with PCA

In [51]:
# Gradient Boosting with Cluster 3 and PCA:
c3_pca_train = c3_train[feat]
c3_pca_test = c3_test[feat]
grid_para_tree = [{'learning_rate': [0.005, 0.01, 0.015, 0.02],
                  'subsample'    : [0.6, 0.5, 0.4, 0.2],
                  'n_estimators' : [100, 500, 1000]
#                  'max_depth'    : [2,5,10]
                 }]

boost3_pca = RandomizedSearchCV(gbm, grid_para_tree, 
                                cv = 3, 
                                n_jobs = -1, n_iter = 48)

boost3_pca.fit(c3_pca_train, Y_c3train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=3,
                   estimator=GradientBoostingRegressor(learning_rate=0.01,
                                                       n_estimators=500,
                                                       subsample=0.7),
                   n_iter=48, n_jobs=-1,
                   param_distributions=[{'learning_rate': [0.005, 0.01, 0.015,
                                                           0.02],
                                         'n_estimators': [100, 500, 1000],
                                         'subsample': [0.6, 0.5, 0.4, 0.2]}])

In [52]:
# Results of Cluster 3 and PCA:
print("R2 of train = ", boost3_pca.best_score_)
print("R2 of test = ", boost3_pca.best_estimator_.score(c3_pca_test, Y_c3test))

pred3 = boost3_pca.predict(c3_pca_train)
pred_unlog3 = np.exp(pred3)

pred_test3 = boost3_pca.predict(c3_pca_test)
pred_test_unlog3 = np.exp(pred_test3)

Y_unlog3 = np.exp(Y_c3train)
Y_test_unlog3 = np.exp(Y_c3test)

train_RMSE3 = mean_squared_error(Y_unlog3, pred_unlog3, squared = False)
test_RMSE3 = np.sqrt(mean_squared_error(pred_test_unlog3, Y_test_unlog3))
print("Train RMSE = ", train_RMSE3)
print("Test RMSE = ", test_RMSE3)

R2 of train =  0.9745616824851931
R2 of test =  0.9847049096682389
Train RMSE =  44.33533245459912
Test RMSE =  84.23821789871623


In [53]:
# Create dataframe from results of Cluster 3 and PCA:
r2_train = boost3_pca.best_score_
r2_test = boost3_pca.best_estimator_.score(c3_pca_test, Y_c3test)
rf_output['PCA_Cluster_3'] = [train_RMSE3, test_RMSE3, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,All,Cluster_1,Cluster_2,Cluster_3,PCA_All,PCA_Cluster_1,PCA_Cluster_2,PCA_Cluster_3
RMSE_train,62.668893,50.654001,52.465481,50.078748,63.596086,57.519318,55.018928,44.335332
RMSE_test,76.321628,79.887594,72.383182,85.518178,75.955993,79.225763,72.565419,84.238218
R2_train,0.984016,0.967987,0.981382,0.967021,0.984276,0.969956,0.981583,0.974562
R2_test,0.989243,0.986478,0.985843,0.984048,0.989537,0.986704,0.985766,0.984705


## Pickle all the PCA files

In [54]:
pickle.dump(boost_pca, open('boost_all_pca.pkl','wb'))
pickle.dump(boost1_pca, open('boost_C1_pca.pkl','wb'))
pickle.dump(boost2_pca, open('boost_C2_pca.pkl','wb'))
pickle.dump(boost3_pca, open('boost_C3_pca.pkl','wb'))

## XGBoost

In [17]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state = 0)

model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
pred = model.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = model.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

In [21]:
print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
train_RMSE

print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))


Train RMSE =  20.82557253274334
Test RMSE =  80.54936457612692


In [22]:
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.9990999612909691
0.9873656202440846


In [23]:
hyperparameters = {
    'colsample_bylevel': [0.75, 0.8, 0.85],
    'colsample_bynode': [0.7, 0.8, 0.85],
    'colsample_bytree': [0.75, 0.8, 0.85], 
    'learning_rate': [0.025, 0.05, 0.1, 0.15],
    'max_depth': [5, 6, 7],
    'max_leaves': [5, 6, 7],
    'min_child_weight': [5, 6, 7],
    'subsample': [0.7, 0.75, 0.8],
    'gamma':[0.75, 0.8, 0.85],
    'n_estimators': [150, 200, 250]
}

gs_boost = XGBRegressor(random_state=0, reg_lambda = 0)
model = RandomizedSearchCV(gs_boost, hyperparameters, cv = 2, n_jobs=-1, n_iter = 10)

# model.fit(X_train, Y_train, eval_set = [(X_ver,Y_ver)], early_stopping_rounds = 20)
model.fit(X_train, Y_train) #, early_stopping_rounds = 20)


RandomizedSearchCV(cv=2,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                          validate_parameters=None,
                                          verbosity=None),
                   n_jobs=-1,
                   param_distribu

In [24]:
pred = model.predict(X_train)
pred_unlog = np.exp(pred)

pred_test = model.predict(X_test)
pred_test_unlog = np.exp(pred_test)

Y_unlog = np.exp(Y_train)
Y_test_unlog = np.exp(Y_test)

In [25]:
print("Train RMSE = ", np.sqrt(mean_squared_error(pred_unlog, Y_unlog)))
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
train_RMSE

print("Test RMSE = ", np.sqrt(mean_squared_error(pred_test_unlog, Y_test_unlog)))


Train RMSE =  100.72561995465115
Test RMSE =  112.32663256056514


In [26]:
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.984320567326284
0.9816836165466879


In [27]:
model.best_estimator_
model.best_params_

{'subsample': 0.75,
 'n_estimators': 250,
 'min_child_weight': 6,
 'max_leaves': 7,
 'max_depth': 7,
 'learning_rate': 0.05,
 'gamma': 0.8,
 'colsample_bytree': 0.85,
 'colsample_bynode': 0.7,
 'colsample_bylevel': 0.8}